# Analyze Product Sentiment

In [1]:
import turicreate

# Read product review data

In [2]:
products = turicreate.SFrame('~/venv/Apple ML foundations/amazon_baby.sframe')

# Explore data

In [3]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


In [4]:
products.groupby('name',operations={'count':turicreate.aggregate.COUNT()}).sort('count',ascending=False)

name,count
Vulli Sophie the GiraffeTeether ...,785
"Simple Wishes Hands-FreeBreastpump Bra, Pink, ...",562
Infant Optics DXR-5 2.4GHz Digital Video Baby ...,561
Baby Einstein Take AlongTunes ...,547
Cloud b TwilightConstellation Night ...,520
"Fisher-Price BoosterSeat, Blue/Green/Gray ...",489
Fisher-Price RainforestJumperoo ...,450
"Graco Nautilus 3-in-1 CarSeat, Matrix ...",419
Leachco Snoogle TotalBody Pillow ...,388
"Regalo Easy Step WalkThru Gate, White ...",374


# Examine the reivews for the most-reviewed product

In [5]:
giraffe_reviews = products[products['name']=='Vulli Sophie the Giraffe Teether']

In [6]:
giraffe_reviews

name,review,rating
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3.0


In [7]:
len(giraffe_reviews)

785

In [8]:
giraffe_reviews['rating'].show()

Materializing SArray

# Building a sentiment classifier

## Build word count vectors

In [9]:
products['word_count'] = turicreate.text_analytics.count_words(products['review'])

In [10]:
products

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'handles': 1,'stripping': 1, 'while': ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1, 'moist':1, 'osocozy': 1, 'keps': ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1,'comfortable': 1, 'bed': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1, 'clever': 1,'little': 1, 'loves': 1, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1, 'many': 1,'headaches': 1, 'soo' ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1, 'or': 1,'break': 1, 'trying': 1, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1, 'barnes': 1,'at': 1, 'is': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1, 'because':1, 'questions': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1, 'and': 1,'changes': 1, 'the': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1, 'pages': 1,'out': 1, 'run': 1, ..."


# Define what is positive and negative sentiment

In [11]:
products['rating'].show()

Materializing SArray

In [12]:
#ignore all 3*  reviews
products = products[products['rating']!= 3]

In [13]:
#positive sentiment = 4-star or 5-star reviews
products['sentiment'] = products['rating'] >= 4

In [14]:
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1, 'moist':1, 'osocozy': 1, 'keps': ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1,'comfortable': 1, 'bed': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1, 'clever': 1,'little': 1, 'loves': 1, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1, 'many': 1,'headaches': 1, 'soo' ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1, 'or': 1,'break': 1, 'trying': 1, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1, 'barnes': 1,'at': 1, 'is': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1, 'because':1, 'questions': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1, 'and': 1,'changes': 1, 'the': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1, 'pages': 1,'out': 1, 'run': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1,'recommend': 1, 'high ...",1


In [15]:
products['sentiment'].show()

Materializing SArray

# Train our sentiment classifier

In [90]:
train_data,test_data = products.random_split(.8,seed=0)

In [91]:
sentiment_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['word_count'], validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.669978  | 1.739942     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 2.012254     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 2.517992     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 4.085978     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

# Apply the sentiment classifier to better understand the Giraffe reviews

In [112]:
products['predicted_sentiment'] = sentiment_model.predict(products, output_type = 'probability')

In [113]:
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1, 'moist':1, 'osocozy': 1, 'keps': ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1,'comfortable': 1, 'bed': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1, 'clever': 1,'little': 1, 'loves': 1, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1, 'many': 1,'headaches': 1, 'soo' ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1, 'or': 1,'break': 1, 'trying': 1, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1, 'barnes': 1,'at': 1, 'is': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1, 'because':1, 'questions': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1, 'and': 1,'changes': 1, 'the': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1, 'pages': 1,'out': 1, 'run': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1,'recommend': 1, 'high ...",1


In [20]:
giraffe_reviews = products[products['name']== 'Vulli Sophie the Giraffe Teether']

In [21]:
giraffe_reviews

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0,"{'purchase': 1,'teething': 1, 'cranky': ...",1
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0,"{'a': 1, 'is': 1, 'when':1, 'him': 1, 'help': 1, ...",1
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0,"{'made': 1, 'of': 1,'packaging': 1, 'no': 1, ...",0
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0,"{'another': 1, 'out': 1,'run': 1, 'lost': 1, ...",1
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0,"{'disappointed': 1,'will': 1, 'take': 1, ...",1
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0,"{'late': 1, 'perfect': 1,'pack': 1, 'only': 1, ...",1
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0,"{'delight': 1, 'in': 1,'other': 1, 'hours': 1, ...",1
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0,"{'inside': 1, 'water': 1,'don': 1, 'up': 1, ...",1
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0,"{'its': 1, 'fine': 1,'is': 1, 'which': 1, ...",1
Vulli Sophie the GiraffeTeether ...,This product is without adoubt the best on the ...,5.0,"{'belongs': 1, 'dog': 1,'if': 1, 'beware': 1, ...",1


# Sort the Giraffe reviews according to predicted sentiment

In [22]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [23]:
giraffe_reviews

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5.0,"{'use': 1, 'get': 1,'will': 1, 'where': 1, ...",1
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4.0,"{'explore': 1,'adventures': 1, 'many': ...",1
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5.0,"{'09': 1, '11': 1,'recommended': 1, 'la': ...",1
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5.0,"{'parents': 1,'recommend': 1, 'happy': ...",1
Vulli Sophie the GiraffeTeether ...,"As every mom knows, youalways want to give your ...",5.0,"{'future': 1, 'use': 1,'am': 1, 'chewing': 1, ...",1
Vulli Sophie the GiraffeTeether ...,My Mom-in-Law boughtSophie for my son whe ...,5.0,"{'penny': 1, 'worth': 1,'definitely': 1, 'han ...",1
Vulli Sophie the GiraffeTeether ...,"My 4 month old son isteething, and I've tried ...",4.0,"{'teether': 1, 'want': 1,'price': 1, 'keepsake': ...",1
Vulli Sophie the GiraffeTeether ...,Let me just start off byaddressing the choking ...,5.0,"{'must': 1, 'nuts': 1,'we': 1, 'theirs': 1, ...",1
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4.0,"{'well': 1, 'natural': 1,'makers': 1, 'try': 1, ...",1
Vulli Sophie the GiraffeTeether ...,"I admit, I didn't getSophie the Giraffe at ...",4.0,"{'dye': 1, 'of': 1,'cause': 1, 'fade': 1, ...",1


In [24]:
giraffe_reviews.tail()

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,My 7 1/2 month oldgrandbaby was sittng up ...,1.0,"{'sopie': 1, 'with': 1,'be': 1, 'should': 1, ...",0
Vulli Sophie the GiraffeTeether ...,I was so looking forwardto getting this for my ...,1.0,"{'vulli': 1, 'by': 1,'two': 1, 'garanimals': ...",0
Vulli Sophie the GiraffeTeether ...,Received the product andsmells like cheap rub ...,1.0,"{'was': 1, 'as': 1,'still': 1, 'but': 1, ...",0
Vulli Sophie the GiraffeTeether ...,I got one of these as ashowe gift that my baby ...,1.0,"{'ask': 1, 'off': 1,'rubbed': 1, 'toy': 1, ...",0
Vulli Sophie the GiraffeTeether ...,When we first got it Ithought did my ...,2.0,"{'mom': 1, 'teethers': 1,'btw': 1, 'other': 1, ...",0
Vulli Sophie the GiraffeTeether ...,When I received this thepaint was peeling off in ...,1.0,"{'worth': 1, 'for': 1,'about': 1, 'raves': 1, ...",0
Vulli Sophie the GiraffeTeether ...,"""Sophie the Giraffe"" hastested positive for ...",1.0,"{'they': 1, 'naturally':1, 'occur': 1, 'though': ...",0
Vulli Sophie the GiraffeTeether ...,"My son (now 2.5) LOVEDhis Sophie, and I bought ...",1.0,"{'exaggerated': 1, 'are':1, 'down': 1, 'went': 1, ...",0
Vulli Sophie the GiraffeTeether ...,I wanted to love thisproduct and was excited ...,1.0,"{'ready': 1, 'work': 1,'too': 1, 'just': 1, ...",0
Vulli Sophie the GiraffeTeether ...,This children's toy isnostalgic and very cute. ...,1.0,"{'amazon': 1, 'from': 1,'one': 1, 'by': 1, 'be': ...",0


## Show the most positive reviews

In [25]:
giraffe_reviews[0]['review']

"As a mother of 16month old twins; I bought Sophie [1 for each, of course] when they were 4 months old after careful reading of all reviews. I heard great things about Sophie and wanted to give her a try. At 4 months babies can't do much more than grasp and semi gnaw on Sophie. For many months I had to squeeze Sophie myself [which I personally enjoyed] and set it on their laps. They LOVED Sophie. The squeak is LOUD and sounds exactly like a dog's squeaky chew toy, just for those who are wondering.As they grew and their motor skills developed to each milestone, Sophie gained more and more individual babytime. The twins were able to squeeze her themselves and chew on her around the clock. They love to throw her, stretch her, squeeze her, chew on her, drool on her... you name it, they have done it. One of the two Sophie's took an extended vacation out in the back yard [unbeknownst to me] and once found, a little water had her looking like a champ again... ready to face another day of play

In [26]:
giraffe_reviews[1]['review']

'I\'ll be honest...I bought this toy because all the hip parents seem to have one too and I wanted to be a part of the "hip parent" crowd. The price-tag was somewhat of a deterent but I prevailed and purchased this teether for my daughter.At first, Lily didn\'t know what to make of of Sophie and showed little interest in the polka-dotted creature. I continued to introduce Lily to Sophie and kept the toy in the carrier so that it was on-hand during transitions. Eventually, Lily discovered what a wonderful experience it was to gnaw on the hooves and ears and these two have never been far apart since.Lily really enjoys gumming all the different parts of Sophie like no other teether we have. The size of the toy is great as it is somewhat substantial and so easy for a little one to grasp and hold onto. Lily really enjoys hearing Sophie squeak and will smile whenever Sophie makes a noise or pops her head up from Mommy\'s lap to say hello.People have stopped and commented on Sophie and to the

# Most negative reivews

In [27]:
giraffe_reviews[-1]['review']

"This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking device inside which stopped working after the first couple of days. I returned this item feeling I had overpaid for a toy that was defective and did not meet my expectations. Please do not be swayed by the cute packaging and hype surounding it as I was. One more thing, I was given a full refund from Amazon without any problem."

In [28]:
giraffe_reviews[-2]['review']

'I wanted to love this product and was excited to buy it when I became pregnant but am now hesitant to let my baby use it after reading about the recall in Europe. Apparently, as I understand it, their toxin standards of measurement are lower than ours so they have not been recalled here (apparently we are OK with low levels of nitrates in the toys our children put in their mouths, but Europeans are not...hmmm)...Be that as it may, toxins registering even CLOSE to a dangerous level made me nervous about using. After digging around online I did discover that the company claims to have changed the product after a certain date and lists manufacturing codes so you can check yours (those listed were made after a certain date and are said to be safer). Sadly mine was not made after the &#34;improved&#34; date but I could not return it because there was no formal recall in our country. I considered returning it and hunting for one with an approved manufacturing date but man that was just too 

# Homework

# Task1

In [29]:
##import turicreate

In [30]:
##products = turicreate.SFrame('~/venv/Apple ML foundations/amazon_baby.sframe')

In [31]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [32]:
b=products[1]['word_count'];
b

{'quilt': 1,
 'comfortable': 1,
 'bed': 1,
 'for': 1,
 'anyone': 1,
 'to': 1,
 'would': 1,
 'of': 1,
 'recommend': 1,
 'the': 1,
 'looking': 1,
 'size': 1,
 'type': 1,
 'looks': 1,
 'than': 1,
 'full': 1,
 'it': 1,
 'this': 1,
 'fit': 1,
 'and': 2,
 'perfectly': 1,
 'warmer': 1,
 'very': 1,
 'soft': 1}

In [33]:
def awesome_count(a):
    n=0;
    if 'awesome' in a:
        n=a['awesome'];
    else: 
        n=0;
    return n

In [34]:
def great_count(a):
    n=0;
    if 'great' in a:
        n=a['great'];
    else: 
        n=0;
    return n

In [35]:
def fantastic_count(a):
    n=0;
    if 'fantastic' in a:
        n=a['fantastic'];
    else: 
        n=0;
    return n

In [36]:
def amazing_count(a):
    n=0;
    if 'amazing' in a:
        n=a['amazing'];
    else: 
        n=0;
    return n

In [37]:
def love_count(a):
    n=0;
    if 'love' in a:
        n=a['love'];
    else: 
        n=0;
    return n

In [38]:
def horrible_count(a):
    n=0;
    if 'horrible' in a:
        n=a['horrible'];
    else: 
        n=0;
    return n

In [39]:
def bad_count(a):
    n=0;
    if 'bad' in a:
        n=a['bad'];
    else: 
        n=0;
    return n

In [40]:
def terrible_count(a):
    n=0;
    if 'terrible' in a:
        n=a['terrible'];
    else: 
        n=0;
    return n

In [41]:
def awful_count(a):
    n=0;
    if 'awful' in a:
        n=a['awful'];
    else: 
        n=0;
    return n

In [42]:
def wow_count(a):
    n=0;
    if 'wow' in a:
        n=a['wow'];
    else: 
        n=0;
    return n

In [43]:
def hate_count(a):
    n=0;
    if 'hate' in a:
        n=a['hate'];
    else: 
        n=0;
    return n

In [ ]:
#products['awesome'] = awesome_count(products['word_count'])
#products['great'] = great_count(products['word_count'])
#products['fantastic'] = awesome_count(products['word_count'])
#products['amazing'] = awesome_count(products['word_count'])
#products['love'] = awesome_count(products['word_count'])
#products['horrible'] = awesome_count(products['word_count'])
#products['bad'] = awesome_count(products['word_count'])
#products['terrible'] = awesome_count(products['word_count'])
#products['awful'] = awesome_count(products['word_count'])
#products['wow'] = awesome_count(products['word_count'])
#products['hate'] = awesome_count(products['word_count'])

In [44]:
selected_words

['awesome',
 'great',
 'fantastic',
 'amazing',
 'love',
 'horrible',
 'bad',
 'terrible',
 'awful',
 'wow',
 'hate']

In [45]:
products['awesome'] = products['word_count'].apply(awesome_count);
products['great'] = products['word_count'].apply(great_count);
products['fantastic'] = products['word_count'].apply(fantastic_count);
products['amazing'] = products['word_count'].apply(amazing_count);
products['love'] = products['word_count'].apply(love_count);
products['horrible'] = products['word_count'].apply(horrible_count);
products['bad'] = products['word_count'].apply(bad_count);
products['terrible'] = products['word_count'].apply(terrible_count);
products['awful'] = products['word_count'].apply(awful_count);
products['wow'] = products['word_count'].apply(wow_count);
products['hate'] = products['word_count'].apply(hate_count);

In [46]:
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1, 'moist':1, 'osocozy': 1, 'keps': ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1,'comfortable': 1, 'bed': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1, 'clever': 1,'little': 1, 'loves': 1, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1, 'many': 1,'headaches': 1, 'soo' ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1, 'or': 1,'break': 1, 'trying': 1, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1, 'barnes': 1,'at': 1, 'is': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1, 'because':1, 'questions': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1, 'and': 1,'changes': 1, 'the': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1, 'pages': 1,'out': 1, 'run': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1,'recommend': 1, 'high ...",1


In [47]:
for i in selected_words:
    print(i, products[i].sum())

awesome 3892
great 55791
fantastic 1664
amazing 2628
love 41994
horrible 1110
bad 4183
terrible 1146
awful 687
wow 425
hate 1107


In [ ]:
#products[i]=products['word_count'].apply(f)

In [55]:
products['sentiment']

dtype: int
Rows: 166752
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... ]

## Task2

In [59]:
train_data,test_data = products.random_split(.8,seed=0)

In [61]:
awesome_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['awesome'], validation_set=test_data);
great_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['great'], validation_set=test_data);
fantastic_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['fantastic'], validation_set=test_data)
amazing_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['amazing'], validation_set=test_data)
love_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['love'], validation_set=test_data)
horrible_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['horrible'], validation_set=test_data)
bad_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['bad'], validation_set=test_data)
terrible_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['terrible'], validation_set=test_data)
awful_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['awful'], validation_set=test_data)
wow_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['wow'], validation_set=test_data)
hate_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['hate'], validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 3        | 0.145124     | 0.841459          | 0.840229            |

| 3         | 4        | 0.202299     | 0.841459          | 0.840229            |

| 4         | 5        | 0.257211     | 0.841459          | 0.840229            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [92]:
selected_words_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=selected_words, validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 0.276639     | 0.847708          | 0.846385            |

| 5         | 6        | 0.337998     | 0.847708          | 0.846385            |

| 6         | 7        | 0.397787     | 0.847708          | 0.846385            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [100]:
selected_words_model.coefficients.sort('value',ascending=False)

name,index,class,value,stderr
love,None,1,1.3592688669225153,0.028068300152099435
(intercept),None,1,1.3365913848877558,0.008929969787656753
awesome,None,1,1.133534666034145,0.08399643983187526
amazing,None,1,1.1000933113660283,0.09954776260465983
fantastic,None,1,0.8858047568814295,0.11167591293399656
great,None,1,0.8630655001196618,0.018955052444377323
wow,None,1,-0.009538236067678897,0.1604641122471166
bad,None,1,-0.9914778800650565,0.03848428664699063
hate,None,1,-1.3484407222463124,0.07715698604297333
awful,None,1,-2.0529082040313513,0.10099735435259259


## Task3

In [99]:
sentiment_model.evaluate(test_data)

{'accuracy': 0.9176975738650012,
 'auc': 0.9342357833151299,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  1397 |
 |      0       |        0        |  3931 |
 |      1       |        1        | 26632 |
 |      1       |        0        |  1344 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.951057941255245,
 'log_loss': 0.33047871872412354,
 'precision': 0.9501587641371436,
 'recall': 0.9519588218472976,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.

In [98]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8463848186404036,
 'auc': 0.6936022046674926,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  371  |
 |      0       |        1        |  4957 |
 |      1       |        1        | 27817 |
 |      1       |        0        |  159  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9157860082304526,
 'log_loss': 0.3962265467087378,
 'precision': 0.8487520595594068,
 'recall': 0.9943165570488991,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+-----+-------+------+
 | threshold |        fpr         | tpr |   p   |  n   |
 +-----------+--------------------+-----+-------+------+
 |    0.0    |        1.0         | 1.0 | 27976 | 532

# Task4

In [122]:
diaper_champ_reviews = products[products['name']== 'Baby Trend Diaper Champ']

In [123]:
diaper_champ_reviews

name,review,rating,word_count,sentiment
Baby Trend Diaper Champ,Ok - newsflash. Diapersare just smelly. We've ...,4.0,"{'convenient': 1, 'can':1, 'you': 1, 'fact': 1, ...",1
Baby Trend Diaper Champ,"My husband and I selectedthe Diaper ""Champ"" ma ...",1.0,"{'try': 1, 'going': 1,'re': 1, 'leaves': 1, ...",0
Baby Trend Diaper Champ,Excellent diaper disposalunit. I used it in ...,5.0,"{'nose': 1, 'for': 2,'investment': 1, 'wise': ...",1
Baby Trend Diaper Champ,We love our diaper champ.It is very easy to use ...,5.0,"{'out': 1, 'pull': 1,'open': 1, 'pail': 1, ...",1
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'winter': 1,'especially': 1, ...",1
Baby Trend Diaper Champ,I waited to review thisuntil I saw how it ...,4.0,"{'mom': 1, 'huge': 1,'special': 1, 'good': 1, ...",1
Baby Trend Diaper Champ,I have had a diaper geniefor almost 4 years since ...,1.0,"{'clean': 1, 'it': 8,'thing': 1, 'years': 1, ...",0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'price': 1, 'worth': 1,'other': 2, 'would': 1, ...",1
Baby Trend Diaper Champ,I am so glad I got theDiaper Champ instead of ...,5.0,"{'best': 1, 'that': 1,'handle': 1, 'flip': 1, ...",1
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'need': 1, 'garbage': 1,'regular': 1, 'part': 1, ...",1


In [124]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [120]:
diaper_champ_reviews

name,review,rating,word_count,sentiment
Baby Trend Diaper Champ,I LOVE LOVE LOVE thisproduct! It is SO much ...,4.0,"{'sorry': 1, 'be': 1,'will': 1, 'any': 1, ...",1
Baby Trend Diaper Champ,"Love it, love it, loveit! This lives up to ...",5.0,"{'parents': 1, 'new': 1,'recommended': 1, 'no': ...",1
Baby Trend Diaper Champ,I received my DiaperChamp at my baby shower ...,5.0,"{'second': 2, 'have': 1,'are': 1, 'pull': 1, ...",1
Baby Trend Diaper Champ,"Let me just say, I LOVETHIS PRODUCT!! I used ...",5.0,"{'fantastic': 1, 'over':1, 'recommended': 1, ...",1
Baby Trend Diaper Champ,I love this diaper paleand wouldn't dream of ...,5.0,"{'easy': 1, 'how': 1,'like': 1, 'on': 1, ...",1
Baby Trend Diaper Champ,Works great - no smells.LOVE that it uses reg ...,5.0,"{'plastic': 1, 'in': 1,'each': 1, 'wraps': 1, ...",1
Baby Trend Diaper Champ,I have been using thisdiaper pail for 41/2 ...,5.0,"{'recommend': 1, 'would':1, 'with': 1, 'odors' ...",1
Baby Trend Diaper Champ,I have a two-year-old sonand I love the Diaper ...,5.0,"{'close': 1, 'sticky': 1,'tightly': 1, 'up': 2, ...",1
Baby Trend Diaper Champ,Love the Diaper Champ. Ihad planned to get the ...,4.0,"{'want': 1, 'wouldn': 1,'items': 1, 'this': 1, ...",1
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'need': 1, 'garbage': 1,'regular': 1, 'part': 1, ...",1


In [116]:
products['predicted_sentiment2'] = selected_words_model.predict(products, output_type = 'probability')

In [118]:
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1, 'moist':1, 'osocozy': 1, 'keps': ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1,'comfortable': 1, 'bed': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1, 'clever': 1,'little': 1, 'loves': 1, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1, 'many': 1,'headaches': 1, 'soo' ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1, 'or': 1,'break': 1, 'trying': 1, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1, 'barnes': 1,'at': 1, 'is': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1, 'because':1, 'questions': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1, 'and': 1,'changes': 1, 'the': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1, 'pages': 1,'out': 1, 'run': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1,'recommend': 1, 'high ...",1


In [117]:
diaper_champ_reviews

name,review,rating,word_count,sentiment
Baby Trend Diaper Champ,I LOVE LOVE LOVE thisproduct! It is SO much ...,4.0,"{'sorry': 1, 'be': 1,'will': 1, 'any': 1, ...",1
Baby Trend Diaper Champ,"Love it, love it, loveit! This lives up to ...",5.0,"{'parents': 1, 'new': 1,'recommended': 1, 'no': ...",1
Baby Trend Diaper Champ,I received my DiaperChamp at my baby shower ...,5.0,"{'second': 2, 'have': 1,'are': 1, 'pull': 1, ...",1
Baby Trend Diaper Champ,"Let me just say, I LOVETHIS PRODUCT!! I used ...",5.0,"{'fantastic': 1, 'over':1, 'recommended': 1, ...",1
Baby Trend Diaper Champ,I love this diaper paleand wouldn't dream of ...,5.0,"{'easy': 1, 'how': 1,'like': 1, 'on': 1, ...",1
Baby Trend Diaper Champ,Works great - no smells.LOVE that it uses reg ...,5.0,"{'plastic': 1, 'in': 1,'each': 1, 'wraps': 1, ...",1
Baby Trend Diaper Champ,I have been using thisdiaper pail for 41/2 ...,5.0,"{'recommend': 1, 'would':1, 'with': 1, 'odors' ...",1
Baby Trend Diaper Champ,I have a two-year-old sonand I love the Diaper ...,5.0,"{'close': 1, 'sticky': 1,'tightly': 1, 'up': 2, ...",1
Baby Trend Diaper Champ,Love the Diaper Champ. Ihad planned to get the ...,4.0,"{'want': 1, 'wouldn': 1,'items': 1, 'this': 1, ...",1
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'need': 1, 'garbage': 1,'regular': 1, 'part': 1, ...",1


In [129]:
diaper_champ_reviews[0:1]

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int
	predicted_sentiment	float
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int
	wow	int
	hate	int
	predicted_sentiment2	float

Rows: 1

Data:
+-------------------------+-------------------------------+--------+
|           name          |             review            | rating |
+-------------------------+-------------------------------+--------+
| Baby Trend Diaper Champ | I read a review below that... |  4.0   |
+-------------------------+-------------------------------+--------+
+-------------------------------+-----------+---------------------+---------+-------+
|           word_count          | sentiment | predicted_sentiment | awesome | great |
+-------------------------------+-----------+---------------------+---------+-------+
| {'key': 1, 'have': 1, 'pie... |     1     |  0.9999999999895941 |    0    |   0   |
+-------------------------------+-----------+---------------------+---------+-------+
+-----------+---------+------+----------+-----+----------+-------+-----+------+
| fantastic | amazing | love | horrible | bad | terrible | awful | wow | hate |
+-----------+---------+------+----------+-----+----------+-------+-----+------+
|     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |  0   |
+-----------+---------+------+----------+-----+----------+-------+-----+------+
+----------------------+
| predicted_sentiment2 |
+----------------------+
|  0.7919288370624453  |
+----------------------+
[1 rows x 18 columns]